In [9]:
import pandas as pd
import os
import csv

import re
import datetime 
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor


# Empty Pandas Dataframe

First lets make our empty pandas dataframe to hold our data later on in the program.

In [10]:
wedge_summary = pd.DataFrame()

Now lets add our columns to the empty dataframe

In [11]:
wedge_summary['file_name'] = ''

In [12]:
wedge_summary['num_rows'] = ''

In [13]:
wedge_summary['num_cards'] = ''

In [14]:
wedge_summary['num_dates'] = ''

In [15]:
wedge_summary

,file_name,num_rows,num_cards,num_dates


# Working with Zip Files

Now let's get our zip files from the wedge data into a list for future reference.

In [16]:
## now let me get a list of the zipped files from the Wedge Download

zip_files = os.listdir("holder/")
zip_files

['transArchive_201310_201312_small.zip',
 'transArchive_201207_201209_small.zip',
 'transArchive_201204_201206_inactive_small.zip',
 'transArchive_201304_201306_inactive_small.zip',
 'transArchive_201007_201009_small.zip',
 'transArchive_201105_small.zip',
 'transArchive_201110_201112_small.zip',
 'transArchive_201304_201306_small.zip',
 'transArchive_201404_201406_inactive_small.zip',
 'transArchive_201504_201506_small.zip',
 'transArchive_201612_small.zip',
 'transArchive_201606_small.zip',
 'transArchive_201401_201403_inactive_small.zip',
 'transArchive_201407_201409_small.zip',
 'transArchive_201201_201203_inactive_small.zip',
 'transArchive_201301_201303_inactive_small.zip',
 'transArchive_201310_201312_inactive_small.zip',
 'transArchive_201107_201109_small.zip',
 'transArchive_201601_small.zip',
 'transArchive_201210_201212_inactive_small.zip',
 'transArchive_201010_201012_small.zip',
 'transArchive_201204_201206_small.zip',
 'transArchive_201410_201412_inactive_small.zip',
 'tr

Iterate over the zip files that hold the Wedge transaction files, and use the CSV Sniffer for delimiters.


In [19]:


#now lets make a dict to hold delimeters found in sniffer

delimiter_dict = dict()

#Here I read in my files
for zip in zip_files:
    with ZipFile("holder/" + zip, 'r') as zf :
        files_in_zip = zf.namelist()
        
        #now I can open the files
        for file in files_in_zip:
            open_file = zf.open(file, 'r')
            open_file = io.TextIOWrapper(open_file, encoding = "utf-8")
            
            #initialize the sniffer!
            sniffer = csv.Sniffer().sniff(sample = open_file.readline())
            #print(sniffer.delimiter)
            
            #add results to our dictionary
            delimiter_dict[file] = sniffer.delimiter
                
            open_file.close()
    
           
           
           

In [20]:
delimiter_dict[file]

','

Now let's look for headers

In [21]:
header_dict = dict()
#Here I read in my files
for zip in zip_files:
    with ZipFile("holder/" + zip, 'r') as zf :
        files_in_zip = zf.namelist()
        
        #now I can open the files
        for file in files_in_zip:
            open_file = zf.open(file, 'r')
            open_file = io.TextIOWrapper(open_file, encoding = "utf-8")     #now lets look for headers

            for line in open_file:
                if line[0] == "d" or line[0:2] == '"d':
                    header_dict[file] = "headers"
                    break
                else:
                    header_dict[file] = "no_headers"
                    break
        

In [22]:
header_dict[file]

'no_headers'

Now lets read the files into a pandas dataframe

In [23]:
headers = """datetime  register_no    emp_no    trans_no    upc    description    trans_type    trans_subtype    trans_status    department
quantity    Scale    cost    unitPrice    total    regPrice    altPrice    tax    taxexempt    foodstamp    wicable
discount    memDiscount    discountable    discounttype    voided    percentDiscount    ItemQtty    volDiscType    volume
VolSpecial    mixMatch    matched    memType    staff    numflag    itemstatus    tenderstatus    charflag    varflag
batchHeaderID    local    organic    display    receipt    card_no    store    branch    match_id    trans_id
""".split()

# Reading all Zip Files into Pandas Dataframes

Now put it all together, read in our zips, open the file, check for the delimiter, the presence of headers, and read into PD DF.

In [25]:
final_dfs= []


for zip in zip_files:
    with ZipFile("holder/" + zip, 'r') as zf :
        files_in_zip = zf.namelist()
        
        #now I can open the files
        for file in files_in_zip:
            open_file = zf.open(file, 'r')
            open_file = io.TextIOWrapper(open_file, encoding = "utf-8")
            
            #check open_file's delimeter
            sniffer = csv.Sniffer().sniff(sample = open_file.readline())
            
            #check open_file for headers
            for line in open_file:
                if line[0] == "d" or line[0:2] == '"d':
                    df = pd.read_csv(open_file, sep = sniffer.delimiter)
                    final_dfs.append(df)
                    
                                        
                else:
                    df = pd.read_csv(open_file, sep = sniffer.delimiter, names = headers)
                    final_dfs.append(df)
                    
            
              
            

In [26]:
len(final_dfs)

53

# Filling my Empty Dataframe

Now for each file we need to store a row in wedge summary   

In [27]:
#empty list
cards = []

dates = []

rows =[]


for df in final_dfs:
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    cards.append(df.card_no.nunique())
    dates.append(df['datetime'].dt.date.nunique())
    rows.append(df.shape[0])
    #break
    
#zip_files

wedge_summary["file_name"] = zip_files
wedge_summary["num_cards"] = cards
wedge_summary["num_dates"] = dates
wedge_summary["num_rows"] = rows

In [28]:
#calculate cards

df.card_no.nunique()

413

In [29]:
#calc rows

df.shape

(9999, 50)

In [30]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['datetime'].dt.date.nunique()

1

# Results

In [31]:
#final print out

wedge_summary

,file_name,num_rows,num_cards,num_dates
0,transArchive_201310_201312_small.zip,9999,399,1
1,transArchive_201207_201209_small.zip,9999,356,1
2,transArchive_201204_201206_inactive_small.zip,9999,464,4
3,transArchive_201304_201306_inactive_small.zip,9999,438,7
4,transArchive_201007_201009_small.zip,9999,398,1
5,transArchive_201105_small.zip,9999,334,1
6,transArchive_201110_201112_small.zip,9999,351,1
7,transArchive_201304_201306_small.zip,9999,390,1
8,transArchive_201404_201406_inactive_small.zip,9999,238,19
9,transArchive_201504_201506_small.zip,9999,478,1
